In [1]:
from datetime import datetime, timedelta
import requests
import json
from gtts import gTTS
import tempfile
from pygame import mixer
import transform_stationname2

def say(text, filename=None):
    with tempfile.NamedTemporaryFile(delete=True) as temp:
        tts = gTTS(text, lang='zh-tw')
        if filename is None:
            filename = "{}.mp3".format(temp.name)
        tts.save(filename)
        mixer.init()
        mixer.music.load(filename)
        mixer.music.play()
        while mixer.music.get_busy() == True:
            continue
        mixer.quit()
    
def find_station(input_station):
    station_dict = {
        'NanGang': ['南港', 'NanGang', '南港駅'],
        'TaiPei': ['台北', 'Taipei', '台北駅'],
        'BanQiao': ['板橋', 'Banqiao', '板橋駅'],
        'TaoYuan': ['桃園', 'Taoyuan', '桃園駅'],
        'XinZhu': ['新竹', 'Hsinchu', '新竹駅'],
        'MiaoLi': ['苗栗', 'Miaoli', '苗栗駅'],
        'TaiZhong': ['台中', 'Taichung', '台中駅'],
        'ZhangHua': ['彰化', 'Changhua', '彰化駅'],
        'YunLin': ['雲林', 'Yunlin', '雲林駅'],
        'JiaYi': ['嘉義', 'Chiayi', '嘉義駅'],
        'TaiNan': ['台南', 'Tainan', '台南駅'],
        'ZuoYing': ['左營', 'ZuoYing', '左營駅']
    }
    for key, values in station_dict.items():
        if input_station in values:
            return key
    return None
    
def find_train(StartStation, EndStation, OutWardSearchDate=datetime.now().date(), OutWardSearchTime=datetime.now()):
    # 語音辨識臺台轉換
    if StartStation[0]=="臺":
        StartStation[0] = "台"
    if EndStation[0]=="臺":
        EndStation[0] = "台"
        
    # find station name
    StartStation = find_station(StartStation)
    EndStation = find_station(EndStation)
    
    # data 格式轉換
    OutWardSearchDate = str(OutWardSearchDate).replace("-", "/")
    # 提取小時、分鐘
    current_hour = OutWardSearchTime.hour
    current_minute = OutWardSearchTime.minute
    # 如果分鐘超過 30，小時加 1
    if 0 < current_minute <= 30:
        current_minute = 30
    else:
        current_minute = 0
        current_hour += 1
    time_format = "00"
    OutWardSearchTime = str(current_hour).zfill(len(time_format))+":"+str(current_minute).zfill(len(time_format))
    print(StartStation, EndStation, OutWardSearchDate, OutWardSearchTime)
    
    url = 'https://www.thsrc.com.tw/TimeTable/Search'
    payload = {
        "SearchType": "S",
        "Lang": "TW",
        "StartStation": StartStation,
        "EndStation": EndStation,
        "OutWardSearchDate": OutWardSearchDate,
        "OutWardSearchTime": OutWardSearchTime,
        "ReturnSearchDate": OutWardSearchDate,
        "ReturnSearchTime": OutWardSearchTime,
        "DiscountType": ""
    }  
     
    response = json.loads(requests.post(url, data=payload).text)
    count = 0
    for data in response['data']['DepartureTable']['TrainItem']:
        if data['DepartureTime']>=OutWardSearchTime and data['RunDate']>=OutWardSearchDate:
            count = count + 1
            # upload to server and say
            url = "http://IP_address/main/Railway.php"
            data = {
                'StartStation': StartStation,
                'EndStation': EndStation,
                'TrainNumber': data['TrainNumber'],
                'OutWardSearchDate': OutWardSearchDate,
                'DepartureTime': data['DepartureTime'],
                'DestinationTime': data['DestinationTime'],
                'Duration': data['Duration'],
                'kind': '高鐵'
            } 
            requests.get(url, params=data)
            print('車次: ', data['TrainNumber'], '出發時間: ', data['DepartureTime'], '抵達時間: ', data['DestinationTime'],'車程: ', data['Duration'])
            say("車次"+data['TrainNumber']+"起始站"+StartStation+"終點站"+EndStation+"出發時間"+data['DepartureTime']+"抵達時間"+data['DestinationTime']+"車程需約"+data['Duration'])
        if count>=5:
            break
if __name__ == "__main__":
    find_train('南港', '左營')
    find_train('Banqiao', 'Tainan')
    find_train('新竹', '台北')

NanGang ZuoYing 2023/12/11 20:30
車次:  0687 出發時間:  20:35 抵達時間:  22:45 車程:  02:10
車次:  0861 出發時間:  21:00 抵達時間:  23:25 車程:  02:25
車次:  0165 出發時間:  21:20 抵達時間:  23:05 車程:  01:45
車次:  0693 出發時間:  21:30 抵達時間:  23:40 車程:  02:10
車次:  0333 出發時間:  21:45 抵達時間:  23:55 車程:  02:10
BanQiao TaiNan 2023/12/11 20:30
車次:  0687 出發時間:  20:55 抵達時間:  22:32 車程:  01:37
車次:  0861 出發時間:  21:19 抵達時間:  23:11 車程:  01:52
車次:  0693 出發時間:  21:50 抵達時間:  23:27 車程:  01:37
XinZhu TaiPei 2023/12/11 20:30
車次:  0854 出發時間:  21:08 抵達時間:  21:39 車程:  00:31
車次:  0684 出發時間:  21:25 抵達時間:  21:59 車程:  00:34
車次:  0858 出發時間:  22:08 抵達時間:  22:39 車程:  00:31
車次:  0690 出發時間:  22:30 抵達時間:  23:04 車程:  00:34
車次:  0862 出發時間:  23:12 抵達時間:  23:44 車程:  00:32
